In [3]:
!pip install lenskit 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 37.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 56.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 4.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.40.1
    Uninstalling llvmlite-0.40.1:
      Successfully uninstalled llvmlite-0.40.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
  Attempting uninstall: numba
    Found existing installation: numba 0.57.1
    Uninstalling numba-0.57.1:
      Successfully uninstalled numba-0.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [44]:
import pandas as pd

from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms.item_knn import ItemItem

from sklearn.model_selection import train_test_split, KFold

import numpy as np
import statistics

In [52]:
data_folder = '/kaggle/input/board-games-database-from-boardgamegeek/'

item_ratings_df = pd.read_csv(data_folder + 'user_ratings.csv')
# item_ratings_df = pd.read_csv(data_folder + 'user_ratings_600k.csv')
# item_ratings_df = pd.read_csv(data_folder + 'user_ratings_1.8m.csv')
item_info = pd.read_csv(data_folder + 'games.csv')
item_mecahnics = pd.read_csv(data_folder + 'mechanics.csv')
item_subcategories = pd.read_csv(data_folder + 'subcategories.csv')
item_themes = pd.read_csv(data_folder + 'themes.csv')

In [4]:
# item_ratings_df = item_ratings_df.groupby('Username', group_keys=False).apply(lambda x: x.sample(frac=0.1))

In [53]:
item_ratings_df = item_ratings_df.rename(columns={
    'BGGId':'item',
    'Rating':'rating',
    'Username':'user'
})

In [54]:
item_ratings_df_filtered = item_ratings_df_filtered.drop_duplicates(subset = ['item', 'user'])

In [55]:
user_rating_count = item_ratings_df.groupby('user').count()
user_rating_count

,item,rating
user,,
Fu_Koios,2,2
beastvol,9,9
mycroft,14,14
woh,5,5
(mostly) harmless,1,1
...,...,...
zzzuzu,39,39
zzzvone,21,21
zzzxxxyyy,36,36


In [56]:
min_rating_count = 10
user_rating_count = user_rating_count[user_rating_count['rating'] >= min_rating_count]

In [57]:
user_list_filtered = list(user_rating_count.index)

In [58]:
item_ratings_df_filtered = item_ratings_df[item_ratings_df['user'].isin(user_list_filtered)]

In [59]:
item_ratings_df_filtered

,item,rating,user
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP
...,...,...,...
18942210,165521,3.0,rseater
18942211,165521,3.0,Bluefox86
18942212,165521,3.0,serginator
18942213,193488,1.0,CaptainCattan


### Build

In [60]:
num_recs = 10  # Number of recommendations to generate
item_item = ItemItem(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys_itemitem = Recommender.adapt(item_item)

In [61]:
recsys_itemitem.fit(item_ratings_df_filtered)

In [62]:
 recsys_itemitem.recommend('zzzzzane', 10)

,item,score
0,324856,9.892670
1,275777,9.795043
2,342942,9.745515
3,249277,9.656002
4,345976,9.612687
5,341169,9.597214
6,329082,9.491935
7,323046,9.387120
8,313776,9.334337
9,165694,9.298533


In [14]:
recsys_itemitem.predict_for_user('zzzzzane', [145888, 226522])

item
145888    6.526580
226522    7.780456
dtype: float64

### Test

In [63]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [64]:
kf = KFold(n_splits=5, shuffle=True)

In [65]:
def predict(model, test_dataframe):
    real_ratings = []
    predicted_ratings = []
    
    users = test_dataframe['user'].unique()
    
    for user in users:
        current_frame = test_dataframe[test_dataframe['user'] == user]
        
        items = list(current_frame['item'])
        ratings = list(current_frame['rating'])
        predictions = list(model.predict_for_user(user, items))
        
        real_ratings += ratings
        predicted_ratings += predictions
    
    return real_ratings, predicted_ratings

In [67]:
mae_list = list()
rmse_list = list()
nans_list = list()
non_nans_list = list()

count = 1

for train_index, test_index in kf.split(item_ratings_df_filtered):
    print(f'Doing fold no: {count}')
    count += 1
    
    train_df = item_ratings_df_filtered.iloc[train_index]
    test_df = item_ratings_df_filtered.iloc[test_index]
    test_df = test_df.sample(frac = 0.001)
    
    train_df = train_df.drop_duplicates(subset = ['item', 'user'])
    test_df = test_df.drop_duplicates(subset = ['item', 'user'])
    
    print('Dataframes created')
    
    item_item = ItemItem(10, min_nbrs=3)
    recsys = Recommender.adapt(item_item)
    recsys.fit(train_df)
    print('Model is fitted')
    
    real_ratings, predicted_ratings = predict(recsys, test_df)
    
    y_test = real_ratings
    y_pred = predicted_ratings
    
    y_test_not_nan = list()
    y_pred_not_nan = list()
    
    nans = 0
    non_nans = 0
    
    for i in range(len(y_pred)):
        num = y_pred[i]
        
        if np.isnan(num):
            nans += 1
        else:
            non_nans += 1
            y_test_not_nan.append(y_test[i])
            y_pred_not_nan.append(num)
            
    print(f'nan values: {nans}')
    print(f'non-nan values: {non_nans}')
    
    mae = mean_absolute_error(y_test_not_nan, y_pred_not_nan)
    rmse = mean_squared_error(y_test_not_nan, y_pred_not_nan, squared = False)
    
    mae_list.append(mae)
    rmse_list.append(rmse)
    nans_list.append(nans)
    non_nans_list.append(non_nans)
    
# Compute average results and standard deviations
results = {
    'avg_mae' : statistics.mean(mae_list),
    'stdv_mae' : statistics.stdev(mae_list),
    'avg_rmse' : statistics.mean(rmse_list),
    'stdv_rmse' : statistics.stdev(rmse_list),
    'avg_nans': statistics.mean(nans_list),
    'avg_non_nans' : statistics.mean(non_nans_list)
}

display(results)

Doing fold no: 1
Dataframes created
Model is fitted
nan values: 1
non-nan values: 3674
Doing fold no: 2
Dataframes created
Model is fitted
nan values: 0
non-nan values: 3675
Doing fold no: 3
Dataframes created
Model is fitted
nan values: 0
non-nan values: 3675
Doing fold no: 4
Dataframes created
Model is fitted
nan values: 0
non-nan values: 3675
Doing fold no: 5
Dataframes created
Model is fitted
nan values: 0
non-nan values: 3675


{'avg_mae': 0.849650260667979,
 'stdv_mae': 0.005147369534018413,
 'avg_rmse': 1.139501925417811,
 'stdv_rmse': 0.013279363421001696,
 'avg_nans': 0.2,
 'avg_non_nans': 3674.8}

{'avg_mae': 0.849650260667979,\
 'stdv_mae': 0.005147369534018413,\
 'avg_rmse': 1.139501925417811,\
 'stdv_rmse': 0.013279363421001696,\
 'avg_nans': 0.2,\
 'avg_non_nans': 3674.8}